<a href="https://colab.research.google.com/github/Igor-source/Made_code/blob/main/Descriptors_drugs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
import os
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Descriptors
from rdkit.ML.Descriptors.MoleculeDescriptors import MolecularDescriptorCalculator
import math

In [6]:
from chembl_webresource_client.new_client import new_client

In [9]:
df = pd.read_csv(r"C:\Users\Igorr\Documents\ITMO5grade\Project_with_Susan\Made_code_github\Made_code\All_csv_files\activity_set_filt.csv", index_col = 0)

In [10]:
df

,activity_comment,activity_id,assay_chembl_id,assay_description,assay_type,bao_endpoint,bao_format,bao_label,canonical_smiles,document_chembl_id,...,standard_value,target_chembl_id,target_organism,target_pref_name,target_tax_id,toid,type,units,uo_units,value
0,NaN,748926,CHEMBL810377,In vitro antibacterial activity against the gr...,F,BAO_0002146,BAO_0000218,organism-based format,CO/N=C(\C(=O)N[C@@H]1C(=O)N2C(C(=O)O)=C(COC(N)...,CHEMBL1132087,...,1.00,CHEMBL352,Staphylococcus aureus,Staphylococcus aureus,1280.0,NaN,MIC,ug ml-1,UO_0000274,1.00
1,NaN,748927,CHEMBL810376,In vitro antibacterial activity against the gr...,F,BAO_0002146,BAO_0000218,organism-based format,CO/N=C(\C(=O)N[C@@H]1C(=O)N2C(C(=O)O)=C(COC(N)...,CHEMBL1132087,...,2.00,CHEMBL352,Staphylococcus aureus,Staphylococcus aureus,1280.0,NaN,MIC,ug ml-1,UO_0000274,2.00
2,NaN,748928,CHEMBL883635,In vitro antibacterial activity against the gr...,F,BAO_0002146,BAO_0000218,organism-based format,CO/N=C(\C(=O)N[C@@H]1C(=O)N2C(C(=O)O)=C(COC(N)...,CHEMBL1132087,...,1.00,CHEMBL352,Staphylococcus aureus,Staphylococcus aureus,1280.0,NaN,MIC,ug ml-1,UO_0000274,1.00
3,NaN,748929,CHEMBL810375,In vitro antibacterial activity against the gr...,F,BAO_0002146,BAO_0000218,organism-based format,CO/N=C(\C(=O)N[C@@H]1C(=O)N2C(C(=O)O)=C(COC(N)...,CHEMBL1132087,...,2.00,CHEMBL352,Staphylococcus aureus,Staphylococcus aureus,1280.0,NaN,MIC,ug ml-1,UO_0000274,2.00
4,NaN,748930,CHEMBL810374,In vitro antibacterial activity against the gr...,F,BAO_0002146,BAO_0000218,organism-based format,CO/N=C(\C(=O)N[C@@H]1C(=O)N2C(C(=O)O)=C(COC(N)...,CHEMBL1132087,...,2.00,CHEMBL352,Staphylococcus aureus,Staphylococcus aureus,1280.0,NaN,MIC,ug ml-1,UO_0000274,2.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179590,NaN,20151113,CHEMBL4513082,Antiviral activity determined as inhibition of...,F,BAO_0000201,BAO_0000218,organism-based format,OC[C@@H](O)[C@@H](O)[C@H](O)[C@H](O)CO,CHEMBL4495565,...,0.03,CHEMBL4303835,Severe acute respiratory syndrome coronavirus 2,SARS-CoV-2,2697049.0,NaN,inhibition,%,UO_0000187,0.03
179591,NaN,22071525,CHEMBL4513082,Antiviral activity determined as inhibition of...,F,BAO_0000201,BAO_0000218,organism-based format,OC[C@@H](O)[C@@H](O)[C@H](O)[C@H](O)CO,CHEMBL4495565,...,0.03,CHEMBL4303835,Severe acute respiratory syndrome coronavirus 2,SARS-CoV-2,2697049.0,NaN,inhibition,%,UO_0000187,0.03
179592,NaN,25083065,CHEMBL5256281,Antioxidant activity assessed as hydroxyl radi...,F,BAO_0000375,BAO_0000019,assay format,OC[C@@H](O)[C@@H](O)[C@H](O)[C@H](O)CO,CHEMBL5252641,...,39.00,CHEMBL2362975,NaN,No relevant target,NaN,NaN,Activity,NaN,NaN,39.00
179593,Summarised AC50 (mean value for measurements w...,25197394,CHEMBL5291879,Compound was evaluated for inhibition of rat S...,F,BAO_0000186,BAO_0000019,assay format,OC[C@@H](O)[C@@H](O)[C@H](O)[C@H](O)CO,CHEMBL5291721,...,30000.00,CHEMBL4828,Rattus norvegicus,Synaptic vesicular amine transporter,10116.0,NaN,AC50,uM,UO_0000065,30.00


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 179595 entries, 0 to 179594
Data columns (total 34 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   activity_comment           38607 non-null   object 
 1   activity_id                179595 non-null  int64  
 2   assay_chembl_id            179595 non-null  object 
 3   assay_description          179595 non-null  object 
 4   assay_type                 179595 non-null  object 
 5   bao_endpoint               179595 non-null  object 
 6   bao_format                 179595 non-null  object 
 7   bao_label                  179595 non-null  object 
 8   canonical_smiles           179595 non-null  object 
 9   document_chembl_id         179595 non-null  object 
 10  document_journal           151590 non-null  object 
 11  document_year              151940 non-null  float64
 12  molecule_chembl_id         179595 non-null  object 
 13  molecule_pref_name         179595 

In [17]:
# Выбираем три нужных столбца, например 'A', 'B', и 'C'
dg = df[['molecule_pref_name', 'molecule_chembl_id', 'canonical_smiles']]

# Выводим результат
dg

,molecule_pref_name,molecule_chembl_id,canonical_smiles
0,CEFUROXIME,CHEMBL1436,CO/N=C(\C(=O)N[C@@H]1C(=O)N2C(C(=O)O)=C(COC(N)...
1,CEFUROXIME,CHEMBL1436,CO/N=C(\C(=O)N[C@@H]1C(=O)N2C(C(=O)O)=C(COC(N)...
2,CEFUROXIME,CHEMBL1436,CO/N=C(\C(=O)N[C@@H]1C(=O)N2C(C(=O)O)=C(COC(N)...
3,CEFUROXIME,CHEMBL1436,CO/N=C(\C(=O)N[C@@H]1C(=O)N2C(C(=O)O)=C(COC(N)...
4,CEFUROXIME,CHEMBL1436,CO/N=C(\C(=O)N[C@@H]1C(=O)N2C(C(=O)O)=C(COC(N)...
...,...,...,...
179590,MANNITOL,CHEMBL689,OC[C@@H](O)[C@@H](O)[C@H](O)[C@H](O)CO
179591,MANNITOL,CHEMBL689,OC[C@@H](O)[C@@H](O)[C@H](O)[C@H](O)CO
179592,MANNITOL,CHEMBL689,OC[C@@H](O)[C@@H](O)[C@H](O)[C@H](O)CO
179593,MANNITOL,CHEMBL689,OC[C@@H](O)[C@@H](O)[C@H](O)[C@H](O)CO


In [18]:
dg

,molecule_pref_name,molecule_chembl_id,canonical_smiles
0,CEFUROXIME,CHEMBL1436,CO/N=C(\C(=O)N[C@@H]1C(=O)N2C(C(=O)O)=C(COC(N)...
1,CEFUROXIME,CHEMBL1436,CO/N=C(\C(=O)N[C@@H]1C(=O)N2C(C(=O)O)=C(COC(N)...
2,CEFUROXIME,CHEMBL1436,CO/N=C(\C(=O)N[C@@H]1C(=O)N2C(C(=O)O)=C(COC(N)...
3,CEFUROXIME,CHEMBL1436,CO/N=C(\C(=O)N[C@@H]1C(=O)N2C(C(=O)O)=C(COC(N)...
4,CEFUROXIME,CHEMBL1436,CO/N=C(\C(=O)N[C@@H]1C(=O)N2C(C(=O)O)=C(COC(N)...
...,...,...,...
179590,MANNITOL,CHEMBL689,OC[C@@H](O)[C@@H](O)[C@H](O)[C@H](O)CO
179591,MANNITOL,CHEMBL689,OC[C@@H](O)[C@@H](O)[C@H](O)[C@H](O)CO
179592,MANNITOL,CHEMBL689,OC[C@@H](O)[C@@H](O)[C@H](O)[C@H](O)CO
179593,MANNITOL,CHEMBL689,OC[C@@H](O)[C@@H](O)[C@H](O)[C@H](O)CO


In [19]:
dg.drop_duplicates(inplace=True)

C:\Users\Igorr\AppData\Local\Temp\ipykernel_22012\3725782354.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dg.drop_duplicates(inplace=True)


In [20]:

# Создаем новые индексы
dg.reset_index(drop=True, inplace=True)
dg

,molecule_pref_name,molecule_chembl_id,canonical_smiles
0,CEFUROXIME,CHEMBL1436,CO/N=C(\C(=O)N[C@@H]1C(=O)N2C(C(=O)O)=C(COC(N)...
1,CEFAZOLIN,CHEMBL1435,Cc1nnc(SCC2=C(C(=O)O)N3C(=O)[C@@H](NC(=O)Cn4cn...
2,MINOCYCLINE,CHEMBL1434,CN(C)c1ccc(O)c2c1C[C@H]1C[C@H]3[C@H](N(C)C)C(O...
3,DOXYCYCLINE ANHYDROUS,CHEMBL1433,C[C@H]1c2cccc(O)c2C(=O)C2=C(O)[C@]3(O)C(=O)C(C...
4,CASPOFUNGIN,CHEMBL1630575,CC[C@H](C)C[C@H](C)CCCCCCCCC(=O)N[C@H]1C[C@@H]...
...,...,...,...
89,GEMIFLOXACIN,CHEMBL430,CO/N=C1\CN(c2nc3c(cc2F)c(=O)c(C(=O)O)cn3C2CC2)...
90,AZITHROMYCIN,CHEMBL529,CC[C@H]1OC(=O)[C@H](C)[C@@H](O[C@H]2C[C@@](C)(...
91,AZTREONAM,CHEMBL158,C[C@H]1[C@H](NC(=O)/C(=N\OC(C)(C)C(=O)O)c2csc(...
92,IMIPENEM,CHEMBL148,C[C@@H](O)[C@H]1C(=O)N2C(C(=O)O)=C(SCCNC=N)C[C...


In [21]:
from rdkit.ML.Descriptors.MoleculeDescriptors import MolecularDescriptorCalculator

def get_descriptor_values(mol, descriptors):
    if mol is None:
        # Return a list of NaNs if the molecule is None
        return [np.nan] * len(descriptors)
    calc = MolecularDescriptorCalculator(descriptors)
    ds = calc.CalcDescriptors(mol)
    return ds

def collect_drug_descriptors(dg):

    # Define the descriptor list
    descriptor_list = Descriptors.descList
    descriptors = [descriptor[0] for descriptor in descriptor_list]

    # Initialize a DataFrame to store descriptors
    descriptors_df = pd.DataFrame(columns=descriptors, index=dg.index)

    # Calculate descriptors for each SMILES string
    for i, smile in enumerate(dg["canonical_smiles"]):
        if pd.isna(smile):
            descriptor_values = [np.nan] * len(descriptors)
        else:
            mol = Chem.MolFromSmiles(smile)
            descriptor_values = get_descriptor_values(mol, descriptors)
        descriptors_df.loc[i] = descriptor_values

    # Merge the original drug data with the descriptors
    drug = pd.concat([dg, descriptors_df], axis=1)

    # # Fill missing values with the mean of the column
    # numeric_cols = dg.select_dtypes(include=np.number).columns
    # dg[numeric_cols] = dg[numeric_cols].fillna(dg[numeric_cols].mean())

    return drug

# Assuming `dg` is the input DataFrame that you already have
# Call the function to collect drug descriptors
drug_data = collect_drug_descriptors(dg)
display(drug_data)


[15:03:57] DEPRECATION WARNING: please use MorganGenerator
[15:03:57] DEPRECATION WARNING: please use MorganGenerator
[15:03:57] DEPRECATION WARNING: please use MorganGenerator
[15:03:57] DEPRECATION WARNING: please use MorganGenerator
[15:03:57] DEPRECATION WARNING: please use MorganGenerator
[15:03:57] DEPRECATION WARNING: please use MorganGenerator
[15:03:57] DEPRECATION WARNING: please use MorganGenerator
[15:03:57] DEPRECATION WARNING: please use MorganGenerator
[15:03:57] DEPRECATION WARNING: please use MorganGenerator
[15:03:57] DEPRECATION WARNING: please use MorganGenerator
[15:03:57] DEPRECATION WARNING: please use MorganGenerator
[15:03:57] DEPRECATION WARNING: please use MorganGenerator
[15:03:57] DEPRECATION WARNING: please use MorganGenerator
[15:03:57] DEPRECATION WARNING: please use MorganGenerator
[15:03:57] DEPRECATION WARNING: please use MorganGenerator
[15:03:57] DEPRECATION WARNING: please use MorganGenerator
[15:03:57] DEPRECATION WARNING: please use MorganGenerat

,molecule_pref_name,molecule_chembl_id,canonical_smiles,MaxAbsEStateIndex,MaxEStateIndex,MinAbsEStateIndex,MinEStateIndex,qed,SPS,MolWt,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
0,CEFUROXIME,CHEMBL1436,CO/N=C(\C(=O)N[C@@H]1C(=O)N2C(C(=O)O)=C(COC(N)...,12.570377,12.570377,0.140077,-1.35304,0.295008,21.206897,424.391,...,1,0,0,0,0,0,0,0,0,0
1,CEFAZOLIN,CHEMBL1435,Cc1nnc(SCC2=C(C(=O)O)N3C(=O)[C@@H](NC(=O)Cn4cn...,12.598077,12.598077,0.015313,-1.160962,0.40519,21.0,454.519,...,2,0,0,0,1,0,0,0,0,0
2,MINOCYCLINE,CHEMBL1434,CN(C)c1ccc(O)c2c1C[C@H]1C[C@H]3[C@H](N(C)C)C(O...,13.493332,13.493332,0.001835,-2.62569,0.398372,29.090909,457.483,...,0,0,0,0,0,0,0,0,0,0
3,DOXYCYCLINE ANHYDROUS,CHEMBL1433,C[C@H]1c2cccc(O)c2C(=O)C2=C(O)[C@]3(O)C(=O)C(C...,13.317028,13.317028,0.10372,-2.894779,0.332217,34.375,444.44,...,0,0,0,0,0,0,0,0,0,0
4,CASPOFUNGIN,CHEMBL1630575,CC[C@H](C)C[C@H](C)CCCCCCCCC(=O)N[C@H]1C[C@@H]...,14.58158,14.58158,0.025749,-2.2785,0.047716,28.636364,1093.331,...,0,0,0,0,0,0,0,0,5,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89,GEMIFLOXACIN,CHEMBL430,CO/N=C1\CN(c2nc3c(cc2F)c(=O)c(C(=O)O)cn3C2CC2)...,14.874521,14.874521,0.033885,-1.336542,0.730424,20.892857,389.387,...,0,0,0,0,0,0,0,0,0,0
90,AZITHROMYCIN,CHEMBL529,CC[C@H]1OC(=O)[C@H](C)[C@@H](O[C@H]2C[C@@](C)(...,14.197646,14.197646,0.133072,-1.798815,0.238475,49.615385,748.996,...,0,0,0,0,0,0,0,0,0,0
91,AZTREONAM,CHEMBL158,C[C@H]1[C@H](NC(=O)/C(=N\OC(C)(C)C(=O)O)c2csc(...,12.562142,12.562142,0.034954,-4.756269,0.174788,20.5,435.44,...,0,0,0,0,0,1,0,0,0,0
92,IMIPENEM,CHEMBL148,C[C@@H](O)[C@H]1C(=O)N2C(C(=O)O)=C(SCCNC=N)C[C...,11.938415,11.938415,0.052549,-1.104254,0.224452,26.1,299.352,...,1,0,0,0,0,0,0,0,0,0


In [22]:
drug_data.info(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 94 entries, 0 to 93
Data columns (total 213 columns):
 #    Column                    Dtype 
---   ------                    ----- 
 0    molecule_pref_name        object
 1    molecule_chembl_id        object
 2    canonical_smiles          object
 3    MaxAbsEStateIndex         object
 4    MaxEStateIndex            object
 5    MinAbsEStateIndex         object
 6    MinEStateIndex            object
 7    qed                       object
 8    SPS                       object
 9    MolWt                     object
 10   HeavyAtomMolWt            object
 11   ExactMolWt                object
 12   NumValenceElectrons       object
 13   NumRadicalElectrons       object
 14   MaxPartialCharge          object
 15   MinPartialCharge          object
 16   MaxAbsPartialCharge       object
 17   MinAbsPartialCharge       object
 18   FpDensityMorgan1          object
 19   FpDensityMorgan2          object
 20   FpDensityMorgan3          object

In [26]:
def convert_columns_to_numeric(df, exclude_columns=[]):
    # Применяем pd.to_numeric ко всем столбцам, кроме тех, которые указаны в exclude_columns
    for col in df.columns:
        if col not in exclude_columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')  # Преобразование с заменой ошибок на NaN
    return df

# Пример использования
# Предположим, у тебя есть DataFrame dg, и ты хочешь исключить столбцы "canonical_smiles" и "drug_name" от преобразования
exclude_columns = ['molecule_pref_name','molecule_chembl_id','canonical_smiles']

# Вызываем функцию для преобразования столбцов
drug_descriptors = convert_columns_to_numeric(drug_data, exclude_columns)

# Выводим результат
print(drug_descriptors.dtypes)

molecule_pref_name     object
molecule_chembl_id     object
canonical_smiles       object
MaxAbsEStateIndex     float64
MaxEStateIndex        float64
                       ...   
fr_thiazole             int64
fr_thiocyan             int64
fr_thiophene            int64
fr_unbrch_alkane        int64
fr_urea                 int64
Length: 213, dtype: object


In [27]:
drug_descriptors

,molecule_pref_name,molecule_chembl_id,canonical_smiles,MaxAbsEStateIndex,MaxEStateIndex,MinAbsEStateIndex,MinEStateIndex,qed,SPS,MolWt,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
0,CEFUROXIME,CHEMBL1436,CO/N=C(\C(=O)N[C@@H]1C(=O)N2C(C(=O)O)=C(COC(N)...,12.570377,12.570377,0.140077,-1.353040,0.295008,21.206897,424.391,...,1,0,0,0,0,0,0,0,0,0
1,CEFAZOLIN,CHEMBL1435,Cc1nnc(SCC2=C(C(=O)O)N3C(=O)[C@@H](NC(=O)Cn4cn...,12.598077,12.598077,0.015313,-1.160962,0.405190,21.000000,454.519,...,2,0,0,0,1,0,0,0,0,0
2,MINOCYCLINE,CHEMBL1434,CN(C)c1ccc(O)c2c1C[C@H]1C[C@H]3[C@H](N(C)C)C(O...,13.493332,13.493332,0.001835,-2.625690,0.398372,29.090909,457.483,...,0,0,0,0,0,0,0,0,0,0
3,DOXYCYCLINE ANHYDROUS,CHEMBL1433,C[C@H]1c2cccc(O)c2C(=O)C2=C(O)[C@]3(O)C(=O)C(C...,13.317028,13.317028,0.103720,-2.894779,0.332217,34.375000,444.440,...,0,0,0,0,0,0,0,0,0,0
4,CASPOFUNGIN,CHEMBL1630575,CC[C@H](C)C[C@H](C)CCCCCCCCC(=O)N[C@H]1C[C@@H]...,14.581580,14.581580,0.025749,-2.278500,0.047716,28.636364,1093.331,...,0,0,0,0,0,0,0,0,5,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89,GEMIFLOXACIN,CHEMBL430,CO/N=C1\CN(c2nc3c(cc2F)c(=O)c(C(=O)O)cn3C2CC2)...,14.874521,14.874521,0.033885,-1.336542,0.730424,20.892857,389.387,...,0,0,0,0,0,0,0,0,0,0
90,AZITHROMYCIN,CHEMBL529,CC[C@H]1OC(=O)[C@H](C)[C@@H](O[C@H]2C[C@@](C)(...,14.197646,14.197646,0.133072,-1.798815,0.238475,49.615385,748.996,...,0,0,0,0,0,0,0,0,0,0
91,AZTREONAM,CHEMBL158,C[C@H]1[C@H](NC(=O)/C(=N\OC(C)(C)C(=O)O)c2csc(...,12.562142,12.562142,0.034954,-4.756269,0.174788,20.500000,435.440,...,0,0,0,0,0,1,0,0,0,0
92,IMIPENEM,CHEMBL148,C[C@@H](O)[C@H]1C(=O)N2C(C(=O)O)=C(SCCNC=N)C[C...,11.938415,11.938415,0.052549,-1.104254,0.224452,26.100000,299.352,...,1,0,0,0,0,0,0,0,0,0


In [ ]:
#  # Fill missing values with the mean of the column
# numeric_cols = drug_data_fillna.select_dtypes(include=np.number).columns
# drug_data_fillna[numeric_cols] = drug_data_fillna[numeric_cols].fillna(drug_data_fillna[numeric_cols].mean())

In [25]:
# # drug_data_fillna.reset_index(drop=True, inplace=True)
# drug_data_fillna

In [29]:
drug_descriptors.to_csv(r'C:\Users\Igorr\Documents\ITMO5grade\Project_with_Susan\Made_code_github\Made_code\All_csv_files\drug_descriptors.csv')

In [30]:
drug_descriptors.to_excel(r'C:\Users\Igorr\Documents\ITMO5grade\Project_with_Susan\Made_code_github\Made_code\All_csv_files\drug_descriptors.xlsx')